# Two-phase model simulations for Eulerian nonlinear elastic and diffusive material


## Authors: Andrea Zafferi, Dirk Peschka

Eulerian model for elastic-diffusive material - Porosity Wave Simulation

This .ipynb file collects the code used for generating the data of Section 5 of the article *Discretisation of Eulerian nonlinear elasticity and diffusion using gradient flows* 


## Running the data-generation notebook

This Jupyter notebook (`.ipynb`) generates the simulation data used in the post-processing and analysis notebook.  
It performs the full numerics and writes solution files to disk.

---

### Requirements

To run this notebook, you need:

- **Python 3**
- **Jupyter Notebook or JupyterLab**
- **FEniCS / DOLFIN**
- Standard Python packages:
- `tqdm` (for progress bars)

The notebook writes output files (`solution*.h5`, `solution*.time`, and parameter files) to the specified output directory (e.g. `output01/`).  
Make sure this directory exists and is writable.

---

### How to run the notebook

From a terminal, navigate to the directory containing the notebook and run:

```bash
jupyter notebook Sim_Section5_v01.ipynb


The following cell defines the material parameters of the model and saves them into a .json file. 

In [3]:
from utilities import *

fout = 'output01/'

Len    = 1.0e4                    # m
rho_f  = 1.0e3                    # kg/m^3
rho_s  = 4.0e0                    # Because of the scaling this is actually the ratio between solid and fluid mass density. For values <1 the fluid is heavier. 
g0     = 1e1                      # m/s^2
kappa  = 1e10                     # Pa
viscf  = 1.0e15                   # Pa·s
viscs  = 1.0e15                   # Pa·s
k_perm = 1e4                      # m^2
mu_s   = 1e8                      # Pa
D_diff = 1e-12                    # m^2/s
V0     = 1.0e-7                   # m/s
eps1   = 1.0e14                   # Pa m^2
eps2   = 1.0e6                    # Pa
tau0   = 1.25e-2                     # Timestep size
# 
# # --- parameters for initial data Gaussian distributed ---
# 
alpha       = 50            # inverse of variance
z0          = 5e-3          # Background porosity
mag         = 5e-2          # Porosity peak value

# --- derived dimensional scales ---
P0     = rho_f * g0 * Len           # Pa
TA     = Len / V0                   # s


Pe             = Len**2 / (D_diff * P0 * TA)
Lambda         = viscf * Len**2 / (TA * k_perm * P0)
Sf             = viscf / (P0 * TA)
Ss             = viscs / (P0 * TA)
E              = mu_s / P0
eps1_hat       = eps1 / (P0 * Len**2)
eps2_hat       = eps2 / (P0)
G              = kappa/P0

pars = {}
pars['Len']      = Len 
pars['rho_f']    = rho_f
pars['rho_s']    = rho_s
pars['g0']       = g0
pars['kappa']    = kappa
pars['viscs']    = viscs
pars['viscf']    = viscf
pars['k_perm']   = k_perm
pars['mu_s']     = mu_s
pars['D_diff']   = D_diff
pars['V0']       = V0
pars['eps1']     = eps1
pars['eps2']     = eps2
pars['alpha']    = alpha
pars['z0']       = z0
pars['mag']      = mag
pars['tau0']     = tau0

pars['Pe']       = Pe
pars['Lambda']   = Lambda
pars['Sf']       = Sf
pars['Ss']       = Ss
pars['E']        = E
pars['eps1_hat'] = eps1_hat
pars['eps2_hat'] = eps2_hat
pars['G']        = G

write_dictionary(fout + 'parameters.json',pars)

The following cell is the main file for simulations. The energy and the biliear forms are functions defined outside the main loop. A function that produces proper initial data for the two displacements based on a given inital datum for the fluid content variable ($\zeta$) is also used. 

In [2]:
from fenics import *
# from mshr import *
# import numpy as np
# from pylab import plt
# import logging
from utilities import *
from ufl import SpatialCoordinate
from tqdm.auto import tqdm
from dolfin import set_log_level, LogLevel 
# from dolfin import *

parameters["form_compiler"]["quadrature_degree"] = 3
set_log_level(LogLevel.WARNING)

# ==========================================
# Energy functional
# ==========================================

def get_energy(us, uf, zeta):
    """
     Compute the energy functional.
    
    Args:
        us: solid displacement field
        uf: fluid displacement field
        zeta: fluid content field
        
    Returns:
        Total energy functional
    """
    d       = 2
    x       = SpatialCoordinate(mesh)
    I       = Identity(d)
    Fs      = inv(I - grad(us))
    Ff      = inv(I - grad(uf))
    Js      = det(Fs)
    Jf      = det(Ff)
    Cs      = Fs.T*Fs / Js
    p       = (zeta - z0) - (Js-1)
    g       = Expression(('0','g0'),g0=1,degree=2) 
    
    Hs      = E/2*(tr(Cs-I))
    Hmix    = G*(p**2)/2 
    Hgr     = inner(rho_s/Js*g,x) + inner(zeta*g,x)
    Hreg    = (eps1_hat/2)*inner(grad(zeta),grad(zeta)) + eps2_hat*zeta*(ln(zeta/z0)-1)
    Htot    = Hs + Hmix + Hgr + Hreg

    return Htot*dx
    
# ==========================================
# Bilinear forms
# ==========================================

def s(us,uf,zeta,dot_us,dot_uf,xius,xiuf):
    """
    Bilinear form for viscous and Darcy dissipation
    """
    I = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    Fs = inv(I-grad(us))
    Js = det(Fs)
    
    s_darcy   = (Lambda*(z0/(zeta))**2)*inner((-Fs*dot_us+Ff*dot_uf),(-Fs*xius + Ff*xiuf))
    ss_stokes = Ss*inner(grad(Fs*dot_us),grad(Fs*xius))
    sf_stokes = Sf*inner(grad(Ff*dot_uf),grad(Ff*xiuf))
    blfs      = (s_darcy + ss_stokes + sf_stokes)

    return blfs

def a(old_zeta,muz,xiz):
    """
    Biliner form related to diffusion
    """
    return (1/Pe)*(old_zeta)*inner(grad(muz),grad(xiz)) # old_zeta 
 
def b(uf,zeta,dot_zeta,duf,dzeta):
    """
    Bilinear form for coupling and advective derivatives
    """
    I  = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    blfb = inner(dzeta,dot_zeta) - inner(grad(dzeta),zeta*Ff*duf)
    return blfb
    
def solve_initial_data(zeta):
    """
    Create proper initial data for the displacement fields.
    
    Args:
        zeta: initial zeta field
    
    Returns:
        Initial displacement field
    """
    Vtemp = FunctionSpace(mesh, MixedElement([FEvec,FEvec]))
    
    qtemp,dqtemp    = Function(Vtemp),TestFunction(Vtemp)
    us,uf    = split(qtemp)
    dus,duf  = split(dqtemp)
    I = Identity(2)
    Ff = inv(I-grad(uf))
    Jf = det(Ff)
    Etemp = get_energy(us, uf, zeta) 
    Etemp += inner(grad(uf),grad(uf))*dx
    
    bc0t = DirichletBC(Vtemp.sub(0).sub(0),Constant(0 ),'on_boundary&&(near(x[0],  0,1e-3)||near(x[0],1,1e-3))')      # left & right
    bc1t = DirichletBC(Vtemp.sub(0).sub(1),Constant(0 ),'on_boundary&&(near(x[1],  0,1e-3)||near(x[1],2,1e-3))')      # top & bottom
    bc2t = DirichletBC(Vtemp.sub(1).sub(0),Constant(0 ),'on_boundary&&(near(x[0],  0,1e-3)||near(x[0],1,1e-3))')      # left & right
    bc3t = DirichletBC(Vtemp.sub(1).sub(1),Constant(0 ),'on_boundary&&(near(x[1],  0,1e-3)||near(x[1],2,1e-3))')      # top & bottom
    
    bct      = [bc0t,bc1t,bc2t,bc3t]

    Restemp =  derivative(Etemp,qtemp,dqtemp)  
    solve(Restemp==0, qtemp, bct,solver_parameters={"newton_solver":{"linear_solver"      : "lu",
                                                    "absolute_tolerance" : 1e-7,
                                                    "maximum_iterations" : 30 }}) 
    return qtemp

def evolve(old_q, dt):
    """
        Solve one time step of the problem.
    
    Args:
        old_q: solution from previous time step
        dt: time step size
        
    Returns:
        Updated solution, the iteration and boolean for successful convergence
        """
    Q = old_q.function_space()
    q,dq                           = Function(Q),TestFunction(Q)
    us,uf,zeta,muz                 = split(q)
    dus,duf,dzeta,dmuz             = split(dq)
    old_us,old_uf,old_zeta,old_muz = split(old_q)
    
    dot_us =   (us-old_us)
    dot_uf =   (uf-old_uf)
    dot_zeta = (zeta-old_zeta)
    dot_muz  = (muz - old_muz)
    
    E = get_energy(us, uf, zeta) 
    
    Res  =   dt*a(old_zeta,muz,dmuz)*dx
    Res +=   b(old_uf,old_zeta,dot_zeta,dot_uf,dmuz)*dx + dt*b(old_uf,old_zeta,dzeta,duf,muz)*dx
    Res += - s(old_us,old_uf,old_zeta,dot_us,dot_uf,dus,duf)*dx
    Res += - dt*derivative(E,q,dq)
    """
    Set homogenous Dirichlet boundary conditions for the two displacement
    and inhomogenous Dirichlet for the fluid content at the top and bottom of the domain
    """
    bc0 = DirichletBC(Q.sub(0).sub(0),Constant(0), 'on_boundary && (near(x[0],0,1e-3) || near(x[0],1,1e-3))')        # left & right
    bc1 = DirichletBC(Q.sub(0).sub(1),Constant(0), 'on_boundary && (near(x[1],0,1e-3) || near(x[1],2,1e-3))')        # top & bottom
    bc2 = DirichletBC(Q.sub(1).sub(0),Constant(0), 'on_boundary && (near(x[0],0,1e-3) || near(x[0],1,1e-3))')        # left & right
    bc3 = DirichletBC(Q.sub(1).sub(1),Constant(0), 'on_boundary && (near(x[1],0,1e-3) || near(x[1],2,1e-3))')        # top & bottom
    bc4 = DirichletBC(Q.sub(2)       ,Constant(z0),'on_boundary && (near(x[1],0,1e-3) || near(x[1],2,1e-3))')        # top & bottom
    
    bc      = [bc0,bc1,bc2,bc3,bc4]

    # Computing the Jacobian of the nonlinear problem
    Jac     = derivative(Res,q)
    # Define variational problem
    problem = NonlinearVariationalProblem(Res,q,bc,Jac)
    solver  = NonlinearVariationalSolver(problem)
    solver.parameters['newton_solver'].update(default_solver_parameters)
    q.assign(old_q)
    
    iterations, converged = solver.solve()
   
    return q,iterations,converged
    

# ==========================================
# Begin of simulation
# ==========================================
    
print("here we go")

fout = 'output01/'

pars = read_dictionary(fout + 'parameters.json')

Len        = pars['Len']   
rho_f      = pars['rho_f'] 
rho_s      = pars['rho_s'] 
g0         = pars['g0']    
kappa      = pars['kappa'] 
viscs      = pars['viscs'] 
viscf      = pars['viscf'] 
k_perm     = pars['k_perm']
mu_s       = pars['mu_s']  
D_diff     = pars['D_diff'] 
V0         = pars['V0']    
eps1       = pars['eps1']  
eps2       = pars['eps2']  
alpha      = pars['alpha'] 
z0         = pars['z0']    
mag        = pars['mag']   
tau0       = pars['tau0']  

Pe        = pars['Pe']      
Lambda    = pars['Lambda']  
Sf        = pars['Sf']       
Ss        = pars['Ss']      
E         = pars['E']       
eps1_hat  = pars['eps1_hat']
eps2_hat  = pars['eps2_hat']
G         = pars['G']     

default_solver_parameters = {
            "linear_solver": "lu",
            "maximum_iterations": 12,
            "relaxation_parameter": 1.0,
            "report": True,
            "absolute_tolerance": 1e-5,
            "relative_tolerance": 1e-5,
            "error_on_nonconvergence": False
}

n_steps       = 201      # --- Number of time steps
l             = 1        # --- Domain width
dt_small      = 1e-6     # --- First time step
it0           = 0
t             = 0
k             = 64       
mesh          = mesh_gen(0.5,0.5,k,l)  # Function imported from utilities.py
x             = SpatialCoordinate(mesh)
printcount    = 0

FEvec         = VectorElement("CG", triangle, 1) 
FEscal        = FiniteElement("CG", triangle, 1) 
FE            = MixedElement([FEvec,FEvec,FEscal,FEscal])


Q             = FunctionSpace(mesh, FE)
zeta0         = Expression('z0+mag*exp(-alpha*((pow((x[0]-0.5),2))+(pow(x[1]-0.5,2))))',z0=z0,alpha=alpha,mag=mag,degree=3)
zi            = project(zeta0,FunctionSpace(mesh,FEscal))

print('prepare initial data')
q_init = solve_initial_data(zi)
print('done')

old_q = Function(Q)
assign(old_q.sub(0),q_init.sub(0))
assign(old_q.sub(1),q_init.sub(1))
assign(old_q.sub(2),project(zi,FunctionSpace(mesh,FEscal)))


dt            = Constant(dt_small)
print('perform small time step')
q,it,conv = evolve(old_q, dt)

old_q.assign(q)
print('done')

output_state(mesh,old_q,fout + 'solution' + str(printcount),it0,t)
print("hmin =", mesh.hmin())

conv_fail = 0
i = it0+1
t = dt_small

with tqdm(total=n_steps, desc="Solving PDE", unit="step") as pbar:  # Function used for visualizing a progress bar
    while i < n_steps:
        dt.assign(tau0)
        q, iters, conv = evolve(old_q, dt)

        """
        Check whether the solver converged and, if not, it repeats the iteration with a halven time step 
        """
        if conv:
            printcount += 1
            output_state(mesh, q, fout + f"solution{printcount}", i, t)
            old_q.assign(q)
            t += tau0
            conv_fail = 0

            tau0 = min(50.0, tau0)

            i += 1          # advance only on success
            pbar.update(1)  # advance the progress bar only on success
            pbar.set_postfix(iters=iters, tau=f"{tau0:.2e}")
        else:
            tqdm.write(f"Convergence failed at step {i+1}/{n_steps}, "
                        f"reducing tau0={tau0:.2e}")
            tau0 *= 0.5
            tau0 = max(1e-7, tau0)
            conv_fail += 1
            pbar.set_postfix_str(f"retry={conv_fail}, tau={tau0:.2e}")

here we go
prepare initial data
done
perform small time step
done
hmin = 0.02209708691207961


Solving PDE:   0%|          | 0/201 [00:00<?, ?step/s]